# Final model

Data can be loaded downloaded [here](https://drive.google.com/drive/folders/1yZI5v3ws3b8GZMl_ACe4TO_qebdS2fUz?usp=sharing). The data is contained in the `srp_raw02.zip` and has to be moved to `/data/raw`.
The resulting folder structure looks like this:
`/data/raw/n`, `/data/raw/o` and `/data/raw/x`.

Please note that the source folder of the `raw` files has been changed...
The data processing follows the same step as preceeding notebooks.

In [ ]:
from os import path

import tensorflow as tf

from src.utils import decode_image_record

processed = path.join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([2], tf.int64)
}
shape = (32, 32, 1)

def decoder(example):
    feature = tf.io.parse_single_example(example, features)
    image = tf.io.parse_tensor(feature['image'], tf.float32)
    image.set_shape(shape)
    # We only want the 'label_idx'. Not the 'angle'.
    label = feature['label'][0]

    return [image, label]


train_dataset = decode_image_record(path.join(processed, 'train.tfrecord'), decoder)
validation_dataset = decode_image_record(path.join(processed, 'validate.tfrecord'), decoder)
test_dataset = decode_image_record(path.join(processed, 'test.tfrecord'), decoder)

model_path = path.join('models', 'symbol_classifier','model.h5')
model = tf.keras.models.load_model(model_path)

print(model.evaluate(train_dataset, steps=100, verbose=0))
print(model.evaluate(validation_dataset, steps=100, verbose=0))
print(model.evaluate(test_dataset, steps=100, verbose=0))